# <u>T16 HOI HUB</u>

## Import Libraries

In [1]:
from ipywidgets import widgets
from IPython.display import display, HTML
import os
import io
import base64
import json
from tqdm.notebook import trange, tqdm ###

## Data Exploration

In [13]:
video_folder = os.listdir('data/video')

list_of_video = {}
for video in video_folder:
    # remove hidden file especially for macOS
    if not video.startswith('.'):
        list_of_video[video] = video

## Create Caption files

In [14]:
# read action and frame object from file
def readCaptionFile(filename,videoName):
    # Opening JSON file
    f = open(filename)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # Closing file
    f.close()
    return data[videoName]["actions"]

# convert frame to time
def convertFrameToTime(frame):
    seconds = int(frame/25)
    minutes = "00"
    if seconds >= 60:
        minutes = str(seconds // 60)
        seconds = seconds % 60
    if len(minutes) == 1:
        minutes = "0" + minutes
    seconds = str(seconds)
    #may need handle hour
    if len(seconds) == 1:
        seconds = "0" + seconds 
    return (minutes + ":" + seconds + ".000")

# read reference text from txt file
def readReferenceFile(refFile):
    referenceDict = {}
    with open(refFile) as f:
        lines = f.readlines()
    for i in lines:
        x = i.split()
        referenceDict[str(x[0])] = x[1]
    return referenceDict

# create caption file
def formatCaptionFile(captionList, reference, captionPath):
    start = "WEBVTT\n\n"
    captions = []
    for i in captionList:
        text = reference[str(i[0])]
        lines = convertFrameToTime(i[1]) + " --> " + convertFrameToTime(i[2]) + "\n" + text + "\n\n"
        captions.append(lines)
    f = open(captionPath, "w")
    f.write(start)
    f.writelines(captions)
    f.close()

In [15]:
video_dropdown = widgets.Dropdown(
    options = list_of_video,
    description = 'Videos',
)

video_src = 'data/video/' + video_dropdown.value

def play_video(video_src,caption_src):
    video = io.open(video_src, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video width="650" height="360" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        <track kind="captions" src={1} srclang="en" label="English" default>
        </video>'''.format(encoded.decode('ascii'),caption_src)))


# video dropdown onchange function
def video_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        change['new']
        global video_src 
        video_src = 'data/video/'+ video_dropdown.value
        
          
# display video dropdown
video_dropdown.observe(video_on_change)
display(video_dropdown) 


Dropdown(description='Videos', options={'P02T01C07.vtt': 'P02T01C07.vtt', 'P02T01C06.vtt': 'P02T01C06.vtt', 'P…

In [8]:
# caption
videoName = video_dropdown.value.split(".")
#location of reference are place at root
ref = readReferenceFile('all_labels.txt')
# may need change the caption path to dynamic
captionPath = "data/video/" + videoName[0] + ".vtt"
# model result file should be some directory, here using root 
captionList = readCaptionFile('smarthome_CS_51.json',videoName[0])
formatCaptionFile(captionList,ref,captionPath)


video = video_src.split('/')[-1]
print("Currently playing : " + video)
play_video(video_src, captionPath)

## Extract i3D features from video

In [2]:
import models

In [3]:
from pathlib import Path
from omegaconf import OmegaConf

In [4]:
# after the desired videos are selected
video_paths: list[str] = ["../data/RGB_Video_MP4/P02T01C06.mp4"]
# TODO: choose a project-relative directory
output_path = "/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/data/RGB_i3d_test"

In [5]:
i3d_defaults = OmegaConf.load(Path("feature_extractor/configs/i3d.yml"))
i3d_config = OmegaConf.merge(i3d_defaults, OmegaConf.create({
    "feature_type": "i3d",
    "streams": "rgb",
    "output_path": output_path,
    "video_paths": video_paths,
    "on_extraction": "save_numpy",
    "stack_size": 16,
    "step_size": 16
}))
extractor = models.ExtractI3D(i3d_config)

In [7]:
for video in tqdm(video_paths, desc="videos extracted"):
    extractor._extract(video)

  0%|          | 0/1 [00:00<?, ?it/s]

## Inference

In [12]:
import torch
# Example: to be modified
# lets say TSU has been selected
from TSU_PDAN import HOI_PDAN
from HOI.smarthome_i3d_per_video import TSU as Dataset
from HOI.smarthome_i3d_per_video import TSU_collate_fn as collate_fn

Random_SEED!!!: 0


In [3]:
# TODO: v-iashin
# lets say TSU smarthome is then selected (in practice, a custom json is generated depending on what specific videos are selected)
val_dataset = Dataset("HOI/data/smarthome_CS_51.json", 'testing', "../data/RGB_i3d_16frames_64000_SSD", 1, 51)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=4,
                                                pin_memory=True, collate_fn=collate_fn)
# val_dataloader.root = args.rgb_root

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 536/536 [00:05<00:00, 95.44it/s]


In [4]:
modelrunner = HOI_PDAN()
modelrunner.PDAN_training_parameters()
modelrunner.model.load_state_dict(torch.load("HOI/PDAN/weight_epoch_0.0002_35"))

/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/HOI/models.py:88: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.key_conv.weight, mode='fan_out')
/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/HOI/models.py:89: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.value_conv.weight, mode='fan_out')
/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/HOI/models.py:90: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.query_conv.weight, mode='fan_out')
/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/HOI/models.py:91: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(self.rel_t

<All keys matched successfully>

In [5]:
result = None
# note: this doesn't appear to show up properly in vscode
with tqdm(val_dataloader, unit='batch') as progressive_loader:
    result = modelrunner.evaluate(progressive_loader)
result

  0%|          | 0/185 [00:00<?, ?batch/s]

/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/.venv/lib/python3.10/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/.venv/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/media/starlight/2c72c05a-ec96-4c96-ba3c-50ae4bc6730b/home/starlight/TSU/ict3104-team16-2022/apmeter.py:108: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  rg = torch.range(1, self.scores.size(0)).float()
/media/starlight/2c72c

val-map: tensor(34.1454)
tensor([46.0981, 53.4290, 44.6538, 51.1127, 43.1801, 46.4834, 37.1899, 31.5516,
        42.9660, 21.3682, 17.6236, 13.5470,  1.1666, 41.1923, 49.6549,  3.8188,
        19.1954, 59.6317,  5.5071, 71.6044, 36.5219, 74.4247, 54.4266,  3.0952,
         0.8215, 62.6654, 35.2815, 45.3404, 59.1798, 16.6004,  7.4304,  0.3018,
        33.3252,  3.7282, 43.4654, 88.7000, 37.3753, 18.4837, 25.2225,  5.6187,
        71.5390, 33.2785, 13.1370, 62.3055,  3.1802, 80.0095, 72.6115, 29.4839,
        18.2610,  3.3478,  1.2794])


tensor(34.1454)

## Training HOI ML Model

In [8]:
train_dataset = Dataset("HOI/data/smarthome_CS_51.json", 'training', "../data/RGB_i3d_16frames_64000_SSD", 1, 51)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=4,
                                                pin_memory=True, collate_fn=collate_fn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 536/536 [00:26<00:00, 19.90it/s]


In [14]:
with trange(0,5, unit='epoch', desc='epochs') as epoch_range:
    with tqdm(train_dataloader, unit='batch', desc='training') as train_loader:
        with tqdm(val_dataloader, unit='batch', desc='validating') as val_loader:
            for model in modelrunner.train(
                train_dataloader=train_loader, 
                val_dataloader=val_loader, 
                epoch_range=epoch_range
            ):
                # reset progress bars for next iteration
                train_loader.reset()
                val_loader.reset()
                # save model snapshot at this epoch
                torch.save(model.state_dict(),'./data/pretrained_model/PDAN/weight_epoch_'+str(modelrunner.start_epoch))
                torch.save(model,'./data/pretrained_model/PDAN/model_epoch_'+str(modelrunner.start_epoch))

  0%|          | 0/5 [00:00<?, ?epoch/s]

training:   0%|          | 0/351 [00:00<?, ?batch/s]

validating:   0%|          | 0/185 [00:00<?, ?batch/s]

train-map: tensor(7.4005)
val-map: tensor(13.6736)
tensor([1.4556e+01, 4.1833e+01, 8.9046e+00, 5.9780e+00, 9.0657e-01, 1.7288e+01,
        1.4202e+00, 5.1075e+00, 1.1310e+01, 4.6842e+00, 4.6598e+00, 1.3473e+00,
        2.7479e-01, 4.9669e+00, 2.2991e+01, 3.9146e+00, 1.4208e+00, 3.0398e+01,
        1.7539e+00, 5.5832e+01, 5.0160e+00, 7.3792e+00, 2.8778e+01, 5.3361e+00,
        1.9476e-01, 3.9621e+01, 2.7845e+01, 1.1070e+01, 3.4978e+01, 1.1449e+00,
        9.7553e-01, 9.0512e-03, 3.6942e+00, 6.9876e-01, 2.2886e+01, 4.5529e+01,
        7.0930e+00, 2.9289e+00, 9.9866e+00, 1.1697e+00, 2.5055e+01, 4.5129e+00,
        2.0219e+01, 5.1237e+01, 5.4919e-01, 3.3501e+01, 2.3606e+01, 2.6887e+01,
        1.5436e+01, 3.7692e-01, 9.5107e-02])
train-map: tensor(10.1407)
val-map: tensor(16.0195)
tensor([1.4440e+01, 4.5139e+01, 1.0591e+01, 1.4382e+01, 1.1897e+00, 1.9334e+01,
        1.2840e+00, 7.4700e+00, 1.1914e+01, 5.6119e+00, 5.1394e+00, 2.2644e+00,
        5.6212e-01, 5.9308e+00, 2.2420e+01, 4.5152e+

## Evaluate